<a href="https://colab.research.google.com/github/iBoxerman/tf-intro/blob/colab/TF_intro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>